In [73]:
%load_ext autoreload
%autoreload 2
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OrdinalEncoder, StandardScaler,OneHotEncoder
from xgboost import XGBClassifier, XGBRegressor
import pathlib
import glob
from sklearn.metrics import accuracy_score,precision_score,median_absolute_error,mean_squared_error
from sklearn.model_selection import train_test_split
from sklearn.base import TransformerMixin, BaseEstimator
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from pickle import dump,load

dir_path = pathlib.Path('train/')
csv_path = dir_path.joinpath('*.csv')


all_files = glob.glob(csv_path.as_posix())
df=None
for f in all_files:
    if  df is None:
        df = pd.read_csv(f)
    else:
        df = pd.concat([df,pd.read_csv(f)])
# df = df.drop(['market cap','volume','operating cash flow'],axis=1)

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [74]:
# With all features
numerical_cols = df.select_dtypes('number').columns.drop('trend_sp')
id_feature = ['Stock_code']
categorical_cols = df.select_dtypes(exclude = ['number','boolean']).columns.drop(['Stock_code'])
target_feature = ['trend_sp']

X = df.drop(target_feature + id_feature,axis=1)
y = df[target_feature]

In [75]:
preprocessor = load(open('model/preprocessor.pkl', 'rb'))
X_test = preprocessor.transform(X)

In [76]:
model = XGBRegressor()
model.load_model('model/xgboost_0311.json')

y_test_hat = model.predict(X_test)

In [80]:
k=30

topK = df.iloc[y_test_hat.argsort()[-k:]]

In [81]:
print(topK.trend_sp.describe())

count     30.000000
mean      64.446333
std       26.051129
min       36.860000
25%       49.552500
50%       58.310000
75%       69.090000
max      169.800000
Name: trend_sp, dtype: float64


In [77]:
topK.Stock_code.to_csv('predictions/pred.csv',index=False,header=False)